# Geochronology Calculations

In [1]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from bokeh.models import Range1d, LinearAxis, ColumnDataSource, LabelSet, Span, Slope, Label, Legend

from scipy.interpolate import CubicSpline
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
output_notebook()

import geochron_apps as gc

Loading BokehJS ...

<center><img src="images/geochronology.png" align="center">
https://www.explainxkcd.com/wiki/index.php/1829:_Geochronology
</center>

The following presentation shows some of the geochronology calculations learned in the Advanced Geochronology class at University of Saskatchewan taught by Bruce Eglington and Camille Partin, 2021. Some of the images in this presentation are taken from lectures given by the instructor.

This notebook contains sample calculations typically used in geochronology. It can be obtained at https://git.cs.usask.ca/msv275/advanced-geochronology.  
It can be cloned through the git command:  
* git clone https://git.cs.usask.ca/msv275/advanced-geochronology.git


## Rb-Sr Calculations

Assume an initial 87Sr/86Sr starting composition of **0.704** and 87Rb/86Sr values of **500.0** for a minerals (let’s say it is biotite) formed at **2000** Ma. 
Calculate the present day 87Sr/86Sr composition.

For this question, we simply need to call our calc_isochron function with different parameters.
* initial = 0.704
* pd_ratio = 500
* decay_const = 1.42 x 10<sup>-11</sup>
* t1 = 2000
* t2 = 0

In [2]:
initial = 0.704
pd_ratio = 500.0
decay_const = 1.42*10**-11
t1 = 2000
print("The present day 87Sr/86Sr composition is {}.".format(gc.calc_t2_daughter(initial, pd_ratio, decay_const, t1)))

The present day 87Sr/86Sr composition is 15.107562488909505.


Assume you have measured present day 87Sr/86Sr and 87Rb/86Sr compositions of **0.73** and **1000.0**, respectively in a mineral like biotite. 
Say you also have an estimate of the initial 87Sr/86Sr composition of the rock from some high-Sr concentration mineral like apatite with a value of **0.704**. 
Calculate the apparent age of the biotite.

In order to calculate age, we just rework the isochron equation to:  


ln(present day/parent-daughter - initial/parent-daughter + 1) / λ  

Now we need a new function:

And we know our paramters:
* est_initial = 0.704
* pd_ratio = 1000
* present_day = 0.73
* decay_const = 1.42 x 10<sup>-11</sup>

In [3]:
est_t1_daughter = 0.704
t2_parent = 1000
t2_daughter = 0.73
decay_const = 1.42*10**-11

print("The apparent age of the biotite is {} Ma.".format(gc.calc_age(est_t1_daughter, t2_parent,
                                                                     t2_daughter, decay_const)))

The apparent age of the biotite is 1.8 Ma.


Repeat this calculation assuming the initial 87Sr/86Sr composition was **0.0**. 
Compare the two ages. 
Is there much difference in age?

For this we simply need to change the value of our initial composition to 0.0.

In [4]:
est_initial = 0.0
print("The apparent age of the biotite is {} Ma.".format(gc.calc_age(est_initial, pd_ratio, present_day, decay_const)))

The apparent age of the biotite is 51.4 Ma.


This is about a 50 Ma year difference! The issue is there is little difference between the initial and present day daughter isotopes.

Take the first bullet and calculate 87Sr/86Sr at **500 Ma**.  

Note: I wasn't sure if this implied the minerals formed at 500 Ma, or the minerals formed at 2000 Ma and the calculation is asking for the ratio at 500 Ma (instead of present day), so I did both!

In [5]:
print("If formed 500 Ma the present day 87Sr/86Sr composition is {}.".format(gc.calc_t2_daughter(est_initial,500,decay_const,500, 0)))
print("If formed 2000 Ma the 500 Ma 87Sr/86Sr composition is {}.".format(gc.calc_t2_daughter(est_initial,500,decay_const,2000,500)))

If formed 500 Ma the present day 87Sr/86Sr composition is 3.5626323789329506.
If formed 2000 Ma the 500 Ma 87Sr/86Sr composition is 10.840930109976554.


Assume an initial 87Sr/86Sr starting composition of **0.704** and 87Rb/86Sr values of **0.1, 10, 100 and 500.0** for a set of rocks and minerals formed at **2000 Ma**. 
Calculate the present day 87Sr/86Sr compositions.

Again, we now have a function for this, so we calculate it in the same we did for Pb/Pb

In [6]:
initial = 0.704
pd_list = [0.1,10,100,500.0]
Rb_Sr = []
decay_const = 1.42*10**-11
t1 = 2000
for pd_ratio in pd_list:
    Rb_Sr.append(gc.calc_t2_daughter(initial, pd_ratio, decay_const, t1,))
RbSr_df = pd.DataFrame()
RbSr_df['87Sr/86Sr'] = pd_list
RbSr_df['87Rb/86Sr'] = Rb_Sr
RbSr_df

,87Sr/86Sr,87Rb/86Sr
0,0.1,0.706881
1,10.0,0.992071
2,100.0,3.584712
3,500.0,15.107562


Assume the pluton was metamorphosed at **1000 Ma** and the various rocks and minerals were homogenised. 
Take a simple average of the calculated 87Sr/86Sr at this time as the start of growth of the new mineral systems. 
Assume the newly formed minerals have 87Rb/86Sr values of **1.0, 5.0, 50.0 and 400.0.**
* Calculate present day values. 
* Calculate the slope of the original minerals and rocks and do the same for the metamorphic minerals. 
* Express these slopes in terms of age. 
* What are the initial ratios of the regression lines?

First we build our dataframe for the original minerals, and calculate the composition of 87Sr/86Sr at 1000 Ma. 

In [7]:
df1 = pd.DataFrame()
initial1 = 0.704
decay_const = 1.42*10**-11
t1, t2 = 2000, 1000
df1['1000_Ma_87Rb_86Sr'] = [0.1,10,100,500.0]
df1['1000_Ma_87Sr_86Sr'] = gc.calc_t2_daughter(initial1, df1['1000_Ma_87Rb_86Sr'], decay_const, t1, t2)
print(df1)

   1000_Ma_87Rb_86Sr  1000_Ma_87Sr_86Sr
0                0.1           0.705451
1               10.0           0.849058
2              100.0           2.154583
3              500.0           7.956913


The average 87Sr/86Sr at 1000 Ma is easy to calculate (which will be used for the metamorphic minerals):

In [8]:
avg = df1["1000_Ma_87Sr_86Sr"].mean()
print(avg)

2.9165011204448024


We calculate our slope from the isochron equation, where the slope = *e*<sup>λ x t1</sup> - *e*<sup>λ x t2</sup>

In [9]:
slope1 = np.exp(1.42*10**-11*t1*1000000) - np.exp(1.42*10**-11*t2*1000000)
print(slope1)

0.014505826064217686


Finally let's plot the data in Bokeh!

In [10]:
figure3 = gc.get_figure("Rubidium Strontium Isochron", "87Sr/86Sr", "87Rb/86Sr", [0,550], [0,9])
figure3.circle(df1['1000_Ma_87Rb_86Sr'], df1['1000_Ma_87Sr_86Sr'], color="red")
reg_line = Slope(gradient=slope1, y_intercept=initial1, line_color="red", line_dash="dashed")
figure3.add_layout(reg_line)
hline = Span(location=initial, dimension='width', line_color="grey")
figure3.renderers.extend([hline])
t_text = " t = {} Ma ".format(t2)
i_text = " 87Sr/86Sr initial = {} ".format(initial1)
t_label = Label(x=25, y=8, text=t_text, border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
i_label = Label(x=25, y=7.6, text=i_text, border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
figure3.add_layout(t_label)
figure3.add_layout(i_label)

In [11]:
show(figure3)

We'll repeat the same methods for the metamorphic minerals with the average 87Sr/86Sr calculated above.

In [12]:
df2 = pd.DataFrame()
t1, t2 = 1000, 0
initial2 = avg
df2['0_Ma_87Rb_86Sr'] = [1.0,5.0,50.0,400.0]
df2['0_Ma_87Sr_86Sr'] = gc.calc_t2_daughter(initial2, df2['0_Ma_87Rb_86Sr'], decay_const, t1)
df2

,0_Ma_87Rb_86Sr,0_Ma_87Sr_86Sr
0,1.0,2.930802
1,5.0,2.988008
2,50.0,3.631566
3,400.0,8.637021


We calculate our slope.

In [13]:
slope2 = np.exp(1.42*10**-11*t1*1000000) - np.exp(1.42*10**-11*t2*1000000)
slope2

0.014301298913601324

In [14]:
figure4 = gc.get_figure("Rubidium Strontium Isochron", "87Sr/86Sr", "87Rb/86Sr", [0,550], [2,9])
figure4.circle(df2['0_Ma_87Rb_86Sr'], df2['0_Ma_87Sr_86Sr'], color="red")
reg_line = Slope(gradient=slope2, y_intercept=initial2, line_color="red", line_dash="dashed")
figure4.add_layout(reg_line)
hline = Span(location=initial2, dimension='width', line_color="grey")
figure4.renderers.extend([hline])
t_text = " t = {} Ma ".format(t2)
i_text = " 87Sr/86Sr initial = {} ".format(round(initial2,4))
t_label = Label(x=25, y=8, text=t_text, border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
i_label = Label(x=25, y=7.6, text=i_text, border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
figure4.add_layout(t_label)
figure4.add_layout(i_label)

In [15]:
show(figure4)

What is the MSWD for each of these lines?

There is no information on the weights of each of these samples, so we assume an equal weighting for each sample. Our first step is to calculate what our best fit line predicts our 87Sr/86Sr to be with our given 87Rb/86Sr. This can be done by using the calculation for y with our slope, intercept, and x known. y=mx+b.  
We will also add a column for weights, but for this example we are assuming equal weighting so this will not effect our results.

In [21]:
df1['predicted_1000_Ma_87Sr_86Sr'] = slope1*df1['0_Ma_87Rb_86Sr'] + initial1
df1['weights'] = 1
df1

KeyError: '0_Ma_87Rb_86Sr'

We then calculate the weighted, squared distance from each predicted point to its actual point.

In [17]:
df1['chi_squared'] = ((df1['predicted_1000_Ma_87Sr_86Sr'] - df1['1000_Ma_87Sr_86Sr']) / df1['weights'])**2
df1

,1000_Ma_87Rb_86Sr,1000_Ma_87Sr_86Sr,predicted_1000_Ma_87Sr_86Sr,weights,chi_squared
0,0.1,0.705451,0.705451,1,0.0
1,10.0,0.849058,0.849058,1,0.0
2,100.0,2.154583,2.154583,1,0.0
3,500.0,7.956913,7.956913,1,0.0


And the MSWD is the cumulative sum of these values.

In [18]:
mswd1 = sum(df1['chi_squared'])
mswd1

0.0

Now we just repeat the calculations for the second line.

In [19]:
df2['predicted_0_Ma_87Sr_86Sr'] = slope2*df2['0_Ma_87Rb_86Sr'] + initial2
df2['weights'] = 1
df2['chi_squared'] = ((df2['predicted_0_Ma_87Sr_86Sr'] - df2['0_Ma_87Sr_86Sr']) / df2['weights'])**2
df2

,0_Ma_87Rb_86Sr,0_Ma_87Sr_86Sr,predicted_0_Ma_87Sr_86Sr,weights,chi_squared
0,1.0,2.930802,2.930802,1,0.0
1,5.0,2.988008,2.988008,1,0.0
2,50.0,3.631566,3.631566,1,0.0
3,400.0,8.637021,8.637021,1,0.0


In [20]:
mswd2 = sum(df2['chi_squared'])
mswd2

0.0

This shows that there the MSWD for both lines are 0. This is not unexpected as it is a perfect line.